In [1]:
import sqlite3

# SQLite DB Name
DB_Name =  "devices.db"

# SQLite DB Table Schema
deviceTableSchema="""
DROP TABLE IF EXISTS devices ;
CREATE TABLE devices (
  id integer primary key autoincrement,
  SensorID text,
  tst,
  datetime text,
  lon,
  lat,
  raw
);

CREATE TRIGGER IF NOT EXISTS format_datetime AFTER INSERT ON devices
  BEGIN
    UPDATE devices SET datetime = strftime('%Y-%m-%d %H:%M:%S', DATETIME(NEW.tst, 'unixepoch', 'localtime')) WHERE id=NEW.id;
  END;
"""

# SQLite DB Table Schema
configTableSchema="""
DROP TABLE IF EXISTS config ;
CREATE TABLE config (
  id integer primary key autoincrement,
  SensorID text,
  datetime text,
  company text,
  project text,
  vessel text,
  projection text,
  units text,
  xteType text
);

CREATE TRIGGER IF NOT EXISTS config_datetime AFTER INSERT ON config
  BEGIN
    UPDATE config SET datetime = datetime('now','localtime') WHERE id=NEW.id;
  END;
"""

# SQLite DB Table Schema
calibrateTableSchema="""
DROP TABLE IF EXISTS calibration ;
CREATE TABLE calibration (
  id integer primary key autoincrement,
  SensorID text,
  datetime text,
  project text,
  vessel text,
  length,
  width,
  height,
  cutterLength,
  cutterDraft,
  pinOffX,
  pinOffY,
  pinOffZ,
  gpsOffX,
  gpsOffY,
  gpsOffZ,
  hdgOffX,
  hdgOffY,
  hdgOffZ,
  flipAngleX,
  flipAngleY
);

CREATE TRIGGER IF NOT EXISTS cal_datetime AFTER INSERT ON calibration
  BEGIN
    UPDATE calibration SET datetime = datetime('now','localtime') WHERE id=NEW.id;
  END;
"""

#Connect or Create DB File
conn = sqlite3.connect(DB_Name)
curs = conn.cursor()

#Create Tables
sqlite3.complete_statement(deviceTableSchema)
curs.executescript(deviceTableSchema)

sqlite3.complete_statement(configTableSchema)
curs.executescript(configTableSchema)

sqlite3.complete_statement(calibrateTableSchema)
curs.executescript(calibrateTableSchema)

#Close DB
curs.close()
conn.close()

In [2]:
import paho.mqtt.client as mqtt
import configparser
import time
import datetime
import json
from log_to_db import device_Data_Handler

In [4]:
# Read Config File
config = configparser.ConfigParser()
config.read('mqtt_logger.conf')
config.sections()
projConf = config['PROJECT']
project = projConf['project']

# MQTT Settings
MQTTconf = config['MQTT']
MQTT_Broker = MQTTconf['MQTT_Broker']
MQTT_Port = int(MQTTconf['MQTT_Port'])
Keep_Alive_Interval = int(MQTTconf['Keep_Alive_Interval'])
MQTT_Topic = project + '/#'
username = MQTTconf['username']
password = MQTTconf['password']

In [5]:
#Subscribe to all Sensors at Base Topic
def on_connect(mosq, obj, flags, rc):
    mqttc.subscribe(MQTT_Topic, 0)

#Save Data into DB Table
def on_message(mosq, obj, msg):
    # This is the Master Call for saving MQTT Data into DB
    # For details of "sensor_Data_Handler" function please refer "sensor_data_to_db.py"
    msg.payload = msg.payload.decode('utf-8')
    result=0

    timestamp = datetime.datetime.now()

    print("MQTT Data Received (%s)..." % timestamp)
    print("MQTT Topic: " + msg.topic)
    print("Data: " + msg.payload)

    try:
        result = device_Data_Handler(msg.topic, msg.payload)
    except:
        print("Database log entry unsuccessful.")

    if (result):
        topic = result['project'] + '/' + result['vessel'] + '/import'
        mqttc.publish(topic, payload=json.dumps(result), qos=1, retain=False)

def on_subscribe(mosq, obj, mid, granted_qos):
    pass

def on_publish(client, userdata, mid):
    pass

In [6]:
#create an mqtt client
clientID = "dredgenav-"+"%s" % time.time()
mqttc = mqtt.Client(clientID, clean_session=False)
#mqttc = mqtt.Client(clientID)
mqttc.username_pw_set(username,password)

# Assign event callbacks
mqttc.on_message = on_message
mqttc.on_connect = on_connect
mqttc.on_subscribe = on_subscribe
mqttc.on_publish = on_publish

# Connect
mqttc.connect(MQTT_Broker, int(MQTT_Port), int(Keep_Alive_Interval))

# Continue the network loop
mqttc.loop_forever()

MQTT Data Received (2021-03-23 01:07:12.934158)...
MQTT Topic: brook/brk202/gps
Data: {"tst": 1616490432.0, "nmea": true, "lon": -78.95636166666667, "lat": 36.07998166666667, "t": "u", "_type": "location", "tid": "brk202", "fix_stat": 2, "num_sats": 14, "cog": 145.97, "speed": 0.0, "altitude": 147.0, "geoid_height": -32.9, "hdop": 0.78}
Inserted Data into Devices.

MQTT Data Received (2021-03-23 01:07:12.960120)...
MQTT Topic: brook/brk202/hdg
Data: {"tst": 1616490432.0, "nmea": true, "lon": -78.95643666666666, "lat": 36.07995666666667, "t": "u", "_type": "hdgSensor", "tid": "brk202", "fix_stat": 2, "num_sats": 17, "altitude": 156.1, "geoid_height": -32.9, "hdop": 0.6}
MQTT Data Received (2021-03-23 01:07:12.960539)...
MQTT Topic: brook/brk202/sonar
Data: {"_type": "sonar", "tid": "brk202", "ping": 18518, "depth": 38.919000000000004, "confidence": 0, "pulse_len": 1173, "gain": 6, "start": 0.0, "length": 42.544000000000004, "temp": 29.310000000000002}
MQTT Data Received (2021-03-23 01:0

MQTT Data Received (2021-03-23 01:07:16.598503)...
MQTT Topic: brook/brk202/sonar
Data: {"_type": "sonar", "tid": "brk202", "ping": 18533, "depth": 39.101, "confidence": 0, "pulse_len": 1173, "gain": 6, "start": 0.0, "length": 42.544000000000004, "temp": 29.310000000000002}


KeyboardInterrupt: 